# Import Packages

In [ ]:
import pandas as pd
import numpy as np 
import csv
import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

In [ ]:
path = '/home/manikya_varshney/Documents/Python/Yale/final_h01-20200912-101538.csv'
data = pd.read_csv(path, low_memory=False)

In [ ]:
data.shape

In [ ]:
data

In [ ]:
#data = data[data.columns.values[2]]

In [ ]:
data['extended_tweet_full_text']

#####  1. Casing (Upper or lower case)
##### 2. Noise Removal (Removal of punctuation, white spaces, special characters, HTML tags)
##### 3. Tokenization (Tweets to tokens i.e. words seprated by spaces)
##### 4. Stopword Removal
##### 5. Text Normalization (Stemming and Lemmatization)

In [ ]:
data['extended_tweet_full_text'] = data['extended_tweet_full_text'].str.lower()

In [ ]:
data['extended_tweet_full_text'] = data['extended_tweet_full_text'].str.replace(r"http\S+ | www\S+ | https\S+", "")

In [ ]:
data['extended_tweet_full_text'] = data['extended_tweet_full_text'].str.replace(r'\@\w+', "")

In [ ]:
data['extended_tweet_full_text']

In [ ]:
#tweet = tweet.translate(str.maketrans("", "", string.punctuation))

In [ ]:

#tweet = re.sub(r"http\S+ | www\S+ | https\S+", "", tweet, flags = re.MULTILINE)dd

#tweet = re.sub(r'\@\w+', "", tweet)